<a href="https://colab.research.google.com/github/subham913/BlazeFace_/blob/master/BlazeFace_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
import tensorflow as tf
import pickle
import numpy as np
import cv2
import os

In [0]:
def SingleBlazeBlock(x, filter_size, kernel_size = 5, strides = 1):
  _x = tf.keras.layers.SeparableConv2D(filters = filter_size, kernel_size=kernel_size, strides = strides, padding = 'same', use_bias=False)(x)
  _x = tf.keras.layers.BatchNormalization()(_x)
  channel_pad = _x.shape[-1] - x.shape[-1] #######Channel padding for residual connections
  if(strides==2):
    x = tf.keras.layers.MaxPooling2D()(x)
    if(channel_pad):
      # x = tf.keras.backend.concatenate([x,tf.zeros((x.shape[0],x.shape[1],x.shape[2],channel_pad),dtype = x.dtype)],axis=-1)
      x = tf.keras.backend.concatenate([x,tf.zeros_like(x)],axis=-1) 
  
  x_out = tf.keras.layers.Add()([_x,x])
  x_out = tf.keras.layers.Activation("relu")(x_out)
  return x_out






In [0]:
def DoubleBlazeBlock(x, filter1_size, filter2_size, kernel_size = 5, strides = 1):
  
  _x1 = tf.keras.layers.SeparableConv2D(filters = filter1_size, kernel_size=kernel_size, strides= strides, padding = 'same', use_bias=False)(x)
  _x1 = tf.keras.layers.BatchNormalization()(_x1)
  _x1 = tf.keras.layers.Activation("relu")(_x1)

  _x2 = tf.keras.layers.SeparableConv2D(filters = filter2_size, kernel_size=kernel_size, strides = 1, padding = 'same', use_bias=False)(_x1)
  _x2 = tf.keras.layers.BatchNormalization()(_x2)

  channel_pad = _x2.shape[-1] - x.shape[-1]
  if(strides==2):
    x = tf.keras.layers.MaxPooling2D()(x)
    if(channel_pad):
      # x = tf.keras.backend.concatenate([x,tf.zeros((x.shape[0],x.shape[1],x.shape[2],channel_pad),dtype = x.dtype)],axis=-1)
      x = tf.keras.backend.concatenate([x,tf.zeros_like(x)],axis=-1)
  x_out = tf.keras.layers.Add()([_x2,x])
  x_out = tf.keras.layers.Activation("relu")(x_out)
  return x_out


#Test SingleBlazeBlock

In [5]:
# m  = tf.keras.layers.SeparableConv2D(filters = 24, kernel_size=5, strides = 2, padding = 'same')
a = tf.convert_to_tensor(np.random.randn(3,64,64,24),dtype = tf.float32)
out = SingleBlazeBlock(a, 48, strides=2)
print(out.shape)

(3, 32, 32, 48)


# Test DoubleBlazeBlock

In [6]:
# m  = tf.keras.layers.SeparableConv2D(filters = 24, kernel_size=5, strides = 2, padding = 'same')
a = tf.convert_to_tensor(np.random.randn(3,32,32,48),dtype = tf.float32)
out = DoubleBlazeBlock(a, 24, 48, strides=1)
print(out.shape)

(3, 32, 32, 48)


#Network

In [0]:
def network():
  inputs = tf.keras.layers.Input(shape=(128,128,3))
  x = tf.keras.layers.Conv2D(kernel_size = 5, filters = 24, strides = 2, padding = 'same')(inputs) ## bx64x64x24

  x = SingleBlazeBlock(x, 24) ## bx64x64x24
  
  x = SingleBlazeBlock(x, 24) ## bx64x64x24
  
  x = SingleBlazeBlock(x, 48, strides = 2) ## bx32x32x48
  
  x = SingleBlazeBlock(x, 48) ## bx32x32x48
  
  x = SingleBlazeBlock(x, 48) ## bx32x32x48
  

  x16 = DoubleBlazeBlock(x, 24, 96, strides = 2) ## bx16x16x96
  
  _x = DoubleBlazeBlock(x16, 24, 96) ## bx16x16x96
 
  _x = DoubleBlazeBlock(_x, 24, 96) ## bx16x16x96
  
  _x = DoubleBlazeBlock(_x, 24, 96, strides = 2) ## bx8x8x96
  
  _x = DoubleBlazeBlock(_x, 24, 96) ## bx8x8x96
  
  x8 = DoubleBlazeBlock(_x, 24, 96) ## bx8x8x96
 

  ####confidence
  x16_conf = tf.keras.layers.Conv2D(kernel_size = 3, filters = 2, strides = 1, padding = 'same', activation = 'sigmoid')(x16) ## bx16x16x2
 
  x8_conf = tf.keras.layers.Conv2D(kernel_size = 3, filters = 6, strides = 1, padding = 'same', activation = 'sigmoid')(x8) ## bx8x8x6
 

  x_conf = tf.keras.layers.concatenate([tf.keras.layers.Reshape((512, 1))(x16_conf),tf.keras.layers.Reshape((384, 1))(x8_conf)],axis=1) ## bx896x1

  ###bounding boxes
  x16_bboxes = tf.keras.layers.Conv2D(kernel_size = 3, filters = 8, strides = 1, padding = 'same')(x16) ## bx16x16x8
  
  x8_bboxes = tf.keras.layers.Conv2D(kernel_size = 3, filters = 24, strides = 1, padding = 'same')(x8) ## bx8x8x24
  
  x_bboxes = tf.keras.layers.concatenate([tf.keras.layers.Reshape((512, 4))(x16_bboxes),tf.keras.layers.Reshape((384, 4))(x8_bboxes)],axis=1) ## bx896x4


  #### Finally return both score and bounding boxes


  return tf.keras.models.Model(inputs, [x_conf, x_bboxes])








In [8]:
model = network()
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 64, 64, 24)   1824        input_1[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_3 (SeparableCo (None, 64, 64, 24)   1176        conv2d[0][0]                     
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 64, 64, 24)   96          separable_conv2d_3[0][0]         
______________________________________________________________________________________________

#Testing Model

In [0]:
import tensorflow.keras.backend as K
inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
# print(outputs)
functors = [K.function([inp], [out]) for out in outputs]    # evaluation functions

In [10]:
#Testing
test = np.random.random((8,128,128,3))
for func in functors:
  print(func([test])[0].shape)
# layer_outs = [func([test]) for func in functors]
# print (layer_outs)

(8, 128, 128, 3)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 64, 64, 24)
(8, 32, 32, 24)
(8, 32, 32, 48)
(8, 32, 32, 24)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 32, 32, 48)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 48)
(8, 16, 16, 96)
(8, 16, 16, 48)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 24)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 16, 16, 96)
(8, 8, 8, 24)
(8, 8, 8, 24)
(8, 8, 8, 24)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 24)
(8, 8, 8, 24)
(8, 8, 8, 24)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8, 8, 96)
(8, 8

#Calculate IoU

In [0]:
def get_intersection(bboxA, bboxB):
  '''
  bboxA:[A,4]
  bboxB:[B,4]
  
  returns intersection of the boxes A and B of shape [A,B]
  '''
  numboxes_A = bboxA.shape[0]
  numboxes_B = bboxB.shape[0]
  max_xy = tf.math.minimum(tf.tile(tf.expand_dims(bboxA[:,2:],axis=1), multiples = (1,numboxes_B,1)),tf.tile(tf.expand_dims(bboxB[:,2:],axis=0), multiples = (numboxes_A,1,1)))
  min_xy = tf.math.maximum(tf.tile(tf.expand_dims(bboxA[:,:2],axis=1), multiples = (1,numboxes_B,1)),tf.tile(tf.expand_dims(bboxB[:,:2],axis=0), multiples = (numboxes_A,1,1)))
  intersections = tf.clip_by_value((max_xy - min_xy),clip_value_min = 0, clip_value_max = 16384)
  return intersections[:,:,0]*intersections[:,:,1]

In [0]:
def get_IoU(bboxA, bboxB):
  '''
  bboxA:[A,4]
  bboxB:[B,4]
  bbox_format: (xmin,ymin,xmax,ymax) 

  returns IoU of shape [A,B]
  Note: If bbox is given in (x,y,w,h) format needs to be converted to (xmin,ymin,xmax,ymax)
  '''
  numboxes_A = bboxA.shape[0]
  numboxes_B = bboxB.shape[0]
  area_a = tf.tile(tf.expand_dims((bboxA[:,2]-bboxA[:,0])*(bboxA[:,3]-bboxA[:,1]), axis=1), multiples = (1,numboxes_B))
  area_b = tf.tile(tf.expand_dims((bboxB[:,2]-bboxB[:,0])*(bboxB[:,3]-bboxB[:,1]), axis=0), multiples = (numboxes_A,1))
  intersection = get_intersection(bboxA, bboxB)
  union = area_a + area_b - intersection
  return intersection/union

#Prediction Modules

In [0]:
def preds_to_coordinates(raw_box_preds, priors):
  '''
  raw_box_preds: (b, 896, 4)
  priors: (896,4)
  decodes raw predictions from network to coordinates (xmin, ymin, xmax, ymax)
  '''
  # print(type(raw_box_preds))
  bbox = np.zeros_like(raw_box_preds.numpy())
  xy_center = (raw_box_preds[:,:,:2] / 128) * priors[:,2:] + priors[:,:2] 
  wh = (raw_box_preds[:,:,2:] / 128) * priors[:,2:]
  bbox[:,:,:2] = (xy_center - wh/2).numpy() ####(xmin, ymin)
  bbox[:,:,2:] = (xy_center + wh/2).numpy() ####(xmax, ymax)
  return tf.convert_to_tensor(bbox)







In [0]:
def get_detections_from_preds(raw_score_preds, raw_box_preds, priors, conf_thresh = 0.6):
  '''
  raw_box_preds: (b, 896, 4)
  priors: (896,4)
  raw_score_preds: (b, 896, 1)
  returns detections of shape (b, num_detections, 5)
  '''

  raw_detection_boxes = preds_to_coordinates(raw_box_preds, priors)
  # raw_detection_score = tf.sigmoid(tf.squeeze(tf.clip_by_value(raw_score_preds, clip_value_min = -clip_val, clip_value_max = clip_val),axis=-1))
  raw_detection_score = tf.squeeze(raw_score_preds, axis=-1)
  mask = raw_detection_score >= conf_thresh #### Discard boxes with too low confidence
  # print(mask.shape, type(mask))

  filtered_output = []

  for i in range(raw_box_preds.shape[0]):
    boxes = raw_detection_boxes.numpy()[i, mask[i]]
    scores = raw_detection_score.numpy()[i, mask[i]]
    filtered_output.append(tf.concat([tf.convert_to_tensor(boxes), tf.expand_dims(tf.convert_to_tensor(scores),axis=-1)], axis=-1))

  return filtered_output






#Test get_detections_from_preds module

In [15]:
priors = tf.convert_to_tensor(np.random.randn(896,4),dtype = tf.float32)
raw_score_preds = tf.convert_to_tensor(np.random.randn(8,896,1),dtype = tf.float32)
# print(raw_score_preds)
raw_box_preds = tf.convert_to_tensor(np.random.randn(8,896,4),dtype = tf.float32)
fil_out = get_detections_from_preds(raw_score_preds, raw_box_preds, priors)
print(len(fil_out), fil_out[0])

8 tf.Tensor(
[[-0.8367777  -0.5482367  -0.832338   -0.5485718   1.3645463 ]
 [ 0.761816    0.35967472  0.75716007  0.3601608   0.8008757 ]
 [ 0.6262133  -0.01908054  0.6271714  -0.01190832  1.2597277 ]
 ...
 [-0.8179705  -2.0137105  -0.81657267 -2.0062423   1.9174443 ]
 [-1.1382604   0.9375633  -1.1271114   0.93559575  1.3730727 ]
 [ 0.12569831  0.70423895  0.12703599  0.7020808   0.65729284]], shape=(235, 5), dtype=float32)


In [0]:
def weighted_nms(filtered_boxes, IoU_thresh = 0.5):
  '''
  filtered_boxes:(num_detections,5)
  For each image with overlapping boxes with IoU above
  a certain threshold it weighs the coordinates by the
  normalized confidence score.
  '''
  detection_out = []

  if (len(filtered_boxes) == 0):
     return detection_out

  sorted_ind = tf.argsort(filtered_boxes[:,-1], direction='DESCENDING')

  while(len(sorted_ind)):
    max_score_det = filtered_boxes[sorted_ind[0]]
    max_score_box = tf.expand_dims(max_score_det[:4], axis = 0)
    remaining_boxes = tf.gather_nd(filtered_boxes[:,:4], tf.expand_dims(sorted_ind,axis=-1))
    
    IoUs = get_IoU(max_score_box, remaining_boxes)
    mask = tf.squeeze(IoUs > IoU_thresh, axis=0)
    curr_overlaps = sorted_ind[mask]
    sorted_ind = sorted_ind[~mask]
    
    weighted_detection = max_score_det.numpy()
    if(curr_overlaps.shape[0] > 1):
      score = tf.gather_nd(filtered_boxes[:,4], tf.expand_dims(curr_overlaps,axis=-1))
      coordinates = tf.gather_nd(filtered_boxes[:,:4], tf.expand_dims(curr_overlaps,axis=-1))
      normalized_score = score / tf.math.reduce_sum(score)
      weighted_bbox = tf.math.reduce_sum(coordinates * normalized_score, axis = 0)
      weighted_detection[:4] = weighted_bbox.numpy()
      weighted_detection[4] =  (tf.math.reduce_sum(score) / curr_overlaps.shape[0]).numpy()
    
    detection_out.append(tf.convert_to_tensor(weighted_detection))

  
  return detection_out



  



#Test Weighted NMS

In [17]:
fil_boxes = tf.convert_to_tensor([[0.11111,0.22222, 0.33333, 0.44444, 0.65], [0.222,0.333, 0.444, 0.55, 0.7], [0.11,0.33, 0.44, 0.6, 0.5], [0.1,0.23, 0.44, 0.75, 0.68], [0.222,0.333, 0.444, 0.55, 0.8],[0.2,0.4,0.6,0.8,0.2]])
det = weighted_nms(fil_boxes, IoU_thresh = 0.25)
print(type(det), det)

<class 'list'> [<tf.Tensor: shape=(5,), dtype=float32, numpy=
array([0.19522387, 0.32022387, 0.448597  , 0.4570895 , 0.67      ],
      dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.11111, 0.22222, 0.33333, 0.44444, 0.65   ], dtype=float32)>, <tf.Tensor: shape=(5,), dtype=float32, numpy=array([0.2, 0.4, 0.6, 0.8, 0.2], dtype=float32)>]


In [0]:
def predict(inputs, priors):
  '''
  inputs: tensor of shape (b,H,W,C)

  returns prediction for the input of shape (b,num_detections,5)
  Note: If no face is detected in the image then a tensor of shape (0,5) is added for that image
  '''

  model = network()
  out = model.predict(inputs)
  # print(type(out[0]), type(out[1]))
  raw_detections = get_detections_from_preds(tf.convert_to_tensor(out[0]), tf.convert_to_tensor(out[1]), priors)
  
  final_detections = []

  for i in range(inputs.shape[0]):
    faces = weighted_nms(raw_detections[i])
    if(len(faces)>0):
      faces = tf.stack(faces)
    else:
      faces = tf.zeros([0,5])
    final_detections.append(faces)
  return final_detections

#Test Prediction Module

In [19]:
x = tf.convert_to_tensor(np.zeros((8,128,128,3)),dtype = tf.float32)
priors = tf.convert_to_tensor(np.random.randn(896,4),dtype = tf.float32)
detections = predict(x, priors)
print(len(detections), detections[0].shape)

8 (0, 5)


#Training Modules

In [0]:
def weighted_loss(ground_truth, predictions):
  '''
  ground_truth:[(b,num_priors,4),,(b,num_priors)]
  predictions:[(b,num_priors,4),(b,num_priors,1)]

  Note that we can save priors beforehand
  returns a scalar smooth_l1_loss

  Note: priors can be saved once and for all of shape (896,4) of format (x,y,w,h)
  '''


  matched_loc_offset, matched_conf = ground_truth
  pred_offset, pred_conf = predictions
  batch_size = matched_conf.shape[0]
  num_priors = matched_conf.shape[1]


  pos_class = matched_conf > 0.0
  num_pos = tf.math.reduce_sum(tf.cast(pos_class, dtype = tf.float32), axis=1) ###(b,)

  pred_offset = pred_offset[pos_class]
  matched_loc_offset = matched_loc_offset[pos_class]


  ##### Smooth L1 loss(localisation loss)
  h = tf.keras.losses.Huber() ##### returns averaged loss
  N = tf.math.reduce_sum(num_pos)
  loc_loss = (h(matched_loc_offset, pred_offset) * (batch_size * num_priors * 4))

  ####TODO: Hard Negative Mining 

  ##### Confidence Loss
  bce = tf.keras.losses.BinaryCrossentropy()
  conf_loss = bce(matched_conf, pred_conf) * (batch_size * num_priors)

  averaged_loss = (loc_loss + conf_loss)/N

  return averaged_loss

# Test Weighted Loss

In [21]:
# ground_truth = [tf.convert_to_tensor(np.random.randn(6,896,4),dtype = tf.float32), [tf.convert_to_tensor(np.random.randn(6,896),dtype = tf.float32)]]
ground_truth = [tf.convert_to_tensor(np.random.randn(6,896,4),dtype = tf.float32), tf.ones([6,896],dtype = tf.float32)]
predictions = [tf.convert_to_tensor(np.random.randn(6,896,4),dtype = tf.float32), tf.convert_to_tensor(np.random.randn(6,896,1),dtype = tf.float32)]

loss = weighted_loss(ground_truth, predictions)
print(loss)


tf.Tensor(11.061824, shape=(), dtype=float32)


In [0]:
def get_offsets(matches, priors):
  '''
  matches:(num_priors,4), coordinate format:(xmin,ymin,xmax,ymax)
  priors:(num_priors,4)

  returns offsets of matches to priors of shape (num_priors,4) for regression
  '''

  ghat_cxcy = (matches[:,:2] + matches[:,2:])/(2 * 128) - priors[:,:2]
  ghat_cxcy /= priors[:,2:]

  ghat_wh = tf.math.log((matches[:,2:]-matches[:,:2])/(128*priors[:,2:]))

  return tf.concat([ghat_cxcy, ghat_wh], axis=-1)

In [0]:
def match(gt_box, gt_labels, priors, overlap_thresh = 0.5):
  '''
  gt_box:(num_objects, 4)
  gt_labels:(num_objects)
  matched_loc_offset:tensor of shape (num_priors,4) to be filled with offset for each prior
  matched_conf:tensor of shape (num_priors) to be filled with labels for each prior

  It matches each default box to a ground_truth box. 
  First assign each ground truth box a prior box with max IoU
  and then assign each prior box a ground truth box with IoU greater than 0.5.
  
  '''

  overlaps = get_IoU(gt_box, priors)
  
  #### assigns each gt_box a prior_box with max IoU
  best_prior_overlap = tf.math.reduce_max(overlaps, axis = 1)
  best_prior_idx = tf.math.argmax(overlaps, axis = 1)

  #### assigns each prior_box a gt_box with max IoU
  best_truth_overlap = tf.math.reduce_max(overlaps, axis = 0)
  best_truth_idx = tf.math.argmax(overlaps, axis = 0).numpy() #### assignment to eagertensor is  not possible so convert to numpy 


  #### ensure best match isn't missed due to thresholding
  fill_value = tf.cast(tf.fill(best_prior_idx.shape[0], 3),dtype = best_truth_overlap.dtype)
  best_truth_overlap = tf.tensor_scatter_nd_update(best_truth_overlap, tf.expand_dims(best_prior_idx, axis=1), fill_value)

  for i in range(best_prior_idx.shape[0]):
    best_truth_idx[best_prior_idx[i]] = i
  
  best_truth_idx = tf.convert_to_tensor(best_truth_idx)


  matches = tf.gather_nd(gt_box, tf.expand_dims(best_truth_idx, axis=1))
  labels = (tf.gather_nd(gt_labels, tf.expand_dims(best_truth_idx, axis=1)) + 1).numpy() #### assignment to eagertensor is  not possible so convert to numpy


  bg_mask =  best_truth_overlap < overlap_thresh #### priors with IoU < threshold are background
  labels[bg_mask] = 0 ### background class is 0

  return [get_offsets(matches, priors), tf.convert_to_tensor(labels)]



#Test Matching Strategy

In [24]:
ground_truth_box = tf.convert_to_tensor(np.random.randn(5,4),dtype = tf.float32)
ground_truth_labels = tf.zeros([5],dtype = tf.float32)
priors = tf.convert_to_tensor(np.random.randn(896,4),dtype = tf.float32)


matched_loc_offset, matched_conf = match(ground_truth_box, ground_truth_labels, priors)

print(matched_conf,matched_loc_offset.shape, matched_conf.shape)

tf.Tensor(
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

#Building Dataloader

In [0]:
img_dir = '/home/subh/blazeface_/images'
priors_path = '/home/subh/blazeface_/priors.npy'
labels_path = '/home/subh/blazeface_/labels.pkl'

In [0]:
def read_img(img_path, img_size=(128,128)):
    """
    load image file and rescale it by 255.
    """
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_shape)
    img /= 255.

    return img

In [0]:
def data_generator(overlap_thresh = 0.5, img_size = (128,128)):
  '''
  returns image, bbox_offset_with_labels
  '''

  img_files = os.listdir(img_dir)

  with open(labels_path, 'rb') as f:
    labels = pickle.load(f)

  numofData = len(img_files)
  data_indices = np.arange(numofData)

  priors = tf.convert_to_tensor(np.load(priors_path), dtype=tf.float32)
  
  i=0
  while i<numofData :
    img = get_img(os.path.join(img_dir, img_files[i]))
    labels = labels[i]
    matched_loc_offset,matched_conf = match(labels[:4], labels[4], priors)

    yield matched_loc_offset, matched_conf
    i+=1

In [0]:
gen_data = tf.data.Dataset.from_generator(data_generator,  output_types = (tf.float32,tf.float32), output_shapes = ((896,4),(896,))).shuffle(100).batch(8)